In [ ]:
!pip install sdv

In [ ]:
from sdv.timeseries import PAR
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from pickle import dump, load

In [ ]:
# https://www.kaggle.com/code/xiaxiaxu/predictmachinefailureinadvance/data
sensor = pd.read_csv("sensor.csv")

# only keeping cols w high var in pca analysis + machine status
data = sensor[
    ["timestamp", "sensor_25", "sensor_11", "sensor_36", "sensor_34", "machine_status"]
]

In [ ]:
# okay, what can our machine status be?

# data = data.convert_dtypes()
print(data.dtypes, "\n")
data["timestamp"] = pd.to_datetime(data["timestamp"])

data.machine_status.unique()

In [ ]:
data.loc[data["machine_status"] == "BROKEN"]

In [ ]:
data_around_failures = data.iloc[16000:26000]
len(data_around_failures)

In [ ]:
print(data_around_failures[data_around_failures.isna().any(axis=1)])

# we only have one row with NA, let's simply drop that.

data_around_failures.dropna(axis=0, inplace=True)

print(data_around_failures[data_around_failures.isna().any(axis=1)])

In [ ]:
scaler = MinMaxScaler((-1, 1))
sensor_cols = ["sensor_25", "sensor_11", "sensor_36", "sensor_34"]

# scaling our data, then saving our scaler object for future use.
data_around_failures[sensor_cols] = scaler.fit_transform(
    data_around_failures[sensor_cols]
)

dump(scaler, open("scaler_2.pkl", "wb"))

In [ ]:
data_around_failures.head()

In [ ]:
# finding number of times machine status is diff vars

n = data_around_failures.machine_status.value_counts()
print(n)

In [ ]:
# going to try to keep every 2 mins rather than every min

thin_data = data_around_failures[data_around_failures.index % 2 == 0]
thin_data

In [ ]:
# thin_data = thin_data.convert_dtypes()

print(thin_data.dtypes, "\n")

In [ ]:
sequence_index = "timestamp"
# context_columns = ['machine_status']
context_columns = None

In [ ]:
model = PAR(
    sequence_index=sequence_index,
    context_columns=context_columns,
    verbose=True,
    epochs=256,
)
model.fit(thin_data)

In [ ]:
model.save("model_att_2.pkl")

In [ ]:
sample = model.sample(1)

In [ ]:
import matplotlib.pyplot as plt


thin_data.set_index("timestamp").plot()

In [ ]:
sample.set_index("timestamp").plot()

gave up on the PAR model from SDV after this + consulting with the SDV [slack](https://app.slack.com/client/T017A503B3M/C01GSDFSQ93) - they recommended trying out their CTGAN model which I do in notebook 6.